In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

class SignLanguageDataset(Dataset):
    def __init__(self, csv_file):
        """
        Loads the dataset from a CSV file and preprocesses it.
        - Converts labels to numbers.
        - Normalizes pixel values (0-255) → (0-1).
        """
        df = pd.read_csv(csv_file)

        # Extract labels and features
        self.labels = df.iloc[:, 0].values  # First column is the label
        self.features = df.iloc[:, 1:].values.astype(np.float32) / 255.0  # Normalize pixels

        # Convert labels to numerical values
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)

        # Convert to tensors
        self.features = torch.tensor(self.features, dtype=torch.float32)
        self.labels = torch.tensor(self.labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Load dataset
csv_file = "data.csv"
dataset = SignLanguageDataset(csv_file)

# Split dataset into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [2]:
class SignLanguageClassifier(nn.Module):
    def __init__(self, num_classes):
        super(SignLanguageClassifier, self).__init__()
        self.fc1 = nn.Linear(90000, 512)  # First hidden layer
        self.fc2 = nn.Linear(512, 256)  # Second hidden layer
        self.fc3 = nn.Linear(256, num_classes)  # Output layer

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)  # Dropout to prevent overfitting

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Get number of classes from dataset
num_classes = len(set(dataset.labels.numpy()))

# Initialize model
model = SignLanguageClassifier(num_classes)


In [3]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10  # You can increase this for better performance
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

print("Training complete!")


Epoch [1/10], Loss: 7.6106
Epoch [2/10], Loss: 0.7246
Epoch [3/10], Loss: 0.3541
Epoch [4/10], Loss: 0.2394
Epoch [5/10], Loss: 0.2438
Epoch [6/10], Loss: 0.2234
Epoch [7/10], Loss: 0.1907
Epoch [8/10], Loss: 0.2072
Epoch [9/10], Loss: 0.1646
Epoch [10/10], Loss: 0.1556
Training complete!


In [4]:
# Evaluate model accuracy
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")
torch.save(model.state_dict(),"hand_sign_model.pth")


Test Accuracy: 94.51%
